In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import pymysql
import pickle

In [61]:
con = pymysql.connect(host="127.0.0.1", db="riot", user="root", password="root1234", charset='utf8')
topdata = pd.read_sql("select * from topdata", con)
topdata

D:\kdigital\anaconda\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,tno,blue_top_champ,bt_kill,bt_gold,bt_damageDone,bt_damageTaken,bt_level,bt_minion,red_top_champ,rt_kill,rt_gold,rt_damageDone,rt_damageTaken,rt_level,rt_minion,result
0,1,Gangplank,1,5256,11067,12651,10,93,Fiora,3,4944,11610,15583,11,90,1
1,2,Zac,3,5987,9122,12814,11,114,Renekton,0,5342,8326,13051,10,125,0
2,3,Akali,1,5486,4786,5463,11,129,Fiora,0,4259,3132,6348,10,112,0
3,4,Aatrox,3,5633,6102,9976,10,91,Jax,1,4087,5130,11299,9,50,1
4,5,Sejuani,4,6880,7338,9899,11,119,Nocturne,1,5083,5966,8925,10,84,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29543,29545,Gwen,1,4361,7908,10619,9,90,Volibear,4,6525,9044,11379,11,122,1
29544,29546,Aatrox,2,5799,5159,7760,11,128,Shyvana,0,4241,4455,6399,11,102,0
29545,29547,Nasus,0,3848,3474,10524,9,83,Gwen,2,6460,8118,8260,11,124,1
29546,29548,Gnar,0,4008,5142,8867,10,87,Gangplank,5,7084,8896,9638,12,125,1


In [62]:
# 데이터 타입 확인
topdata = topdata.drop(['tno'],axis=1)
topdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29548 entries, 0 to 29547
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   blue_top_champ  29548 non-null  object
 1   bt_kill         29548 non-null  int64 
 2   bt_gold         29548 non-null  int64 
 3   bt_damageDone   29548 non-null  int64 
 4   bt_damageTaken  29548 non-null  int64 
 5   bt_level        29548 non-null  int64 
 6   bt_minion       29548 non-null  int64 
 7   red_top_champ   29548 non-null  object
 8   rt_kill         29548 non-null  int64 
 9   rt_gold         29548 non-null  int64 
 10  rt_damageDone   29548 non-null  int64 
 11  rt_damageTaken  29548 non-null  int64 
 12  rt_level        29548 non-null  int64 
 13  rt_minion       29548 non-null  int64 
 14  result          29548 non-null  int64 
dtypes: int64(13), object(2)
memory usage: 3.4+ MB


In [63]:
# champ code 불러오기
with open('champ.pickle','rb') as f:
    champCode = pickle.load(f)

champCode[['champion_key','en_name']]

,champion_key,en_name
0,266,Aatrox
1,103,Ahri
2,84,Akali
3,166,Akshan
4,12,Alistar
...,...,...
156,221,Zeri
157,115,Ziggs
158,26,Zilean
159,142,Zoe


In [64]:
# 챔피언명 라벨링
def changeCode(x):
    for code, name in champCode[['champion_key','en_name']].iterrows():
        if x == name[1]:
            return name[0]
red_top_champ_code = topdata['red_top_champ'].apply(changeCode)
blue_top_champ_code = topdata['blue_top_champ'].apply(changeCode)

chmapToCode = pd.concat([red_top_champ_code,blue_top_champ_code],axis=1)
chmapToCode

,red_top_champ,blue_top_champ
0,114,41
1,58,154
2,114,84
3,24,266
4,56,113
...,...,...
29543,106,887
29544,102,266
29545,887,75
29546,41,150


In [65]:
#라벨링된 데이터로변경
labelTop = pd.concat([chmapToCode,topdata.drop(['blue_top_champ','red_top_champ'], axis=1)],axis=1)

#결측치 제거
labelTop = labelTop.dropna(axis=0)

#타입변경
labelTop = labelTop.astype({'red_top_champ':'int64'})
labelTop = labelTop.astype({'blue_top_champ':'int64'})
labelTop.info()
labelTop

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29322 entries, 0 to 29547
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   red_top_champ   29322 non-null  int64
 1   blue_top_champ  29322 non-null  int64
 2   bt_kill         29322 non-null  int64
 3   bt_gold         29322 non-null  int64
 4   bt_damageDone   29322 non-null  int64
 5   bt_damageTaken  29322 non-null  int64
 6   bt_level        29322 non-null  int64
 7   bt_minion       29322 non-null  int64
 8   rt_kill         29322 non-null  int64
 9   rt_gold         29322 non-null  int64
 10  rt_damageDone   29322 non-null  int64
 11  rt_damageTaken  29322 non-null  int64
 12  rt_level        29322 non-null  int64
 13  rt_minion       29322 non-null  int64
 14  result          29322 non-null  int64
dtypes: int64(15)
memory usage: 3.6 MB


,red_top_champ,blue_top_champ,bt_kill,bt_gold,bt_damageDone,bt_damageTaken,bt_level,bt_minion,rt_kill,rt_gold,rt_damageDone,rt_damageTaken,rt_level,rt_minion,result
0,114,41,1,5256,11067,12651,10,93,3,4944,11610,15583,11,90,1
1,58,154,3,5987,9122,12814,11,114,0,5342,8326,13051,10,125,0
2,114,84,1,5486,4786,5463,11,129,0,4259,3132,6348,10,112,0
3,24,266,3,5633,6102,9976,10,91,1,4087,5130,11299,9,50,1
4,56,113,4,6880,7338,9899,11,119,1,5083,5966,8925,10,84,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29543,106,887,1,4361,7908,10619,9,90,4,6525,9044,11379,11,122,1
29544,102,266,2,5799,5159,7760,11,128,0,4241,4455,6399,11,102,0
29545,887,75,0,3848,3474,10524,9,83,2,6460,8118,8260,11,124,1
29546,41,150,0,4008,5142,8867,10,87,5,7084,8896,9638,12,125,1


In [66]:
labelTop = labelTop.reset_index()
labelTop = labelTop.drop(['index'],axis=1)
labelTop

,red_top_champ,blue_top_champ,bt_kill,bt_gold,bt_damageDone,bt_damageTaken,bt_level,bt_minion,rt_kill,rt_gold,rt_damageDone,rt_damageTaken,rt_level,rt_minion,result
0,114,41,1,5256,11067,12651,10,93,3,4944,11610,15583,11,90,1
1,58,154,3,5987,9122,12814,11,114,0,5342,8326,13051,10,125,0
2,114,84,1,5486,4786,5463,11,129,0,4259,3132,6348,10,112,0
3,24,266,3,5633,6102,9976,10,91,1,4087,5130,11299,9,50,1
4,56,113,4,6880,7338,9899,11,119,1,5083,5966,8925,10,84,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29317,106,887,1,4361,7908,10619,9,90,4,6525,9044,11379,11,122,1
29318,102,266,2,5799,5159,7760,11,128,0,4241,4455,6399,11,102,0
29319,887,75,0,3848,3474,10524,9,83,2,6460,8118,8260,11,124,1
29320,41,150,0,4008,5142,8867,10,87,5,7084,8896,9638,12,125,1


In [67]:
topdata = labelTop
# 비주류챔프 삭제
df_red = pd.DataFrame(topdata.red_top_champ.value_counts() < 15).reset_index()
df_blue = pd.DataFrame(topdata.blue_top_champ.value_counts() < 15).reset_index()
# df_red.loc[df_red['red_top_champ'] == True]['index'],df_blue.loc[df_blue['blue_top_champ'] == True]['index']
delList_red = list(df_red.loc[df_red['red_top_champ'] == True]['index'])
delList_blue = list(df_blue.loc[df_blue['blue_top_champ'] == True]['index'])
target_red = []
target_blue = []
for i, row in topdata.iterrows():
    if row['red_top_champ'] in delList_red:
        print(i, row['red_top_champ'])
        target_red.append(i)
for i, row in topdata.iterrows():
    if row['blue_top_champ'] in delList_blue:
        print(i, row['blue_top_champ'])
        target_blue.append(i)
    #if v in delList:
topdata = topdata.drop(index=target_red)   
topdata = topdata.drop(index=target_blue)
topdata

172 427
292 40
559 147
759 246
1037 29
1097 34
1629 203
1649 21
1736 203
2110 96
2291 96
2376 115
2447 37
2623 888
2711 888
2888 20
2889 20
2902 20
3243 115
3270 1
3271 20
3477 136
3674 20
3891 34
3898 1
4484 105
4520 115
4757 28
4906 28
5244 29
5483 201
5486 96
5517 1
5527 246
5566 96
5585 163
5599 29
5909 427
6067 498
6084 498
6268 105
6591 21
6711 888
6972 40
6997 96
7024 40
7103 497
7133 105
7363 163
7403 34
7429 203
7433 1
7509 96
7575 34
7701 63
7954 163
7994 203
8061 28
8098 28
8125 34
8476 498
8477 63
8753 34
8828 96
9259 136
9469 63
9502 63
9653 163
9680 201
9704 246
9856 888
9894 246
9949 163
9963 498
9964 20
9972 61
10185 161
10609 161
10957 203
11032 432
11069 143
11311 21
11545 161
11656 201
11672 29
11726 147
11729 497
11730 432
11833 61
11866 96
11950 63
12008 21
12125 21
12134 61
12135 203
12502 15
12526 147
12683 21
12705 267
12755 63
12961 115
13072 28
13500 163
13633 147
13723 20
13794 498
13877 20
13960 526
13989 163
14123 34
14165 61
14430 20
14490 34
14575 143
145

KeyError: '[5566, 5585, 9964, 11726, 12008, 14575, 14998, 21417, 21424, 23159, 26223, 26567] not found in axis'

In [68]:
topdata = topdata.reset_index()
topdata = topdata.drop(['index'],axis=1)
topdata

,red_top_champ,blue_top_champ,bt_kill,bt_gold,bt_damageDone,bt_damageTaken,bt_level,bt_minion,rt_kill,rt_gold,rt_damageDone,rt_damageTaken,rt_level,rt_minion,result
0,114,41,1,5256,11067,12651,10,93,3,4944,11610,15583,11,90,1
1,58,154,3,5987,9122,12814,11,114,0,5342,8326,13051,10,125,0
2,114,84,1,5486,4786,5463,11,129,0,4259,3132,6348,10,112,0
3,24,266,3,5633,6102,9976,10,91,1,4087,5130,11299,9,50,1
4,56,113,4,6880,7338,9899,11,119,1,5083,5966,8925,10,84,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29084,106,887,1,4361,7908,10619,9,90,4,6525,9044,11379,11,122,1
29085,102,266,2,5799,5159,7760,11,128,0,4241,4455,6399,11,102,0
29086,887,75,0,3848,3474,10524,9,83,2,6460,8118,8260,11,124,1
29087,41,150,0,4008,5142,8867,10,87,5,7084,8896,9638,12,125,1


In [55]:
# 컬럼을 줄여 보았으나 정학도가 오히려 떨어짐
# topdata.loc[topdata['bt_kill'] > topdata['rt_kill'], 'kill'] = 0
# topdata.loc[topdata['bt_kill'] <= topdata['rt_kill'], 'kill'] = 1

# topdata.loc[topdata['bt_gold'] > topdata['rt_gold'], 'gold'] = 0
# topdata.loc[topdata['bt_gold'] <= topdata['rt_gold'], 'gold'] = 1

# topdata.loc[topdata['bt_damageDone'] > topdata['rt_damageDone'], 'damageDone'] = 0
# topdata.loc[topdata['bt_damageDone'] <= topdata['rt_damageDone'], 'damageDone'] = 1

# topdata.loc[topdata['bt_damageTaken'] > topdata['rt_damageTaken'], 'damageTaken'] = 0
# topdata.loc[topdata['bt_damageTaken'] <= topdata['rt_damageTaken'], 'damageTaken'] = 1

# topdata.loc[topdata['bt_level'] > topdata['rt_level'], 'level'] = 0
# topdata.loc[topdata['bt_level'] <= topdata['rt_level'], 'level'] = 1

# topdata.loc[topdata['bt_minion'] > topdata['rt_minion'], 'minion'] = 0
# topdata.loc[topdata['bt_minion'] <= topdata['rt_minion'], 'minion'] = 1

# topdata = topdata.astype('int')
# topdata = topdata.drop(['bt_kill','bt_gold','bt_damageDone','bt_damageTaken','bt_level','bt_minion','rt_kill','rt_gold','rt_damageDone','rt_damageTaken','rt_level','rt_minion'],axis=1)

# topdata = topdata[['red_top_champ', 'blue_top_champ', 'kill', 'gold','damageDone', 'damageTaken', 'level', 'minion', 'result']]
# topdata

,red_top_champ,blue_top_champ,result,kill,gold,damageDone,damageTaken,level,minion
0,114,41,1,1,0,1,1,1,0
1,58,154,0,0,0,0,1,0,1
2,114,84,0,0,0,0,1,0,0
3,24,266,1,0,0,0,1,0,0
4,56,113,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
29084,106,887,1,1,1,1,1,1,1
29085,102,266,0,0,0,0,0,1,0
29086,887,75,1,1,1,1,0,1,1
29087,41,150,1,1,1,1,1,1,1


## 데이터분석

In [20]:
from sklearn.model_selection import train_test_split

In [379]:
# 데이터 분할
x_train, x_test, y_train, y_test=train_test_split(topdata.drop('result',axis=1), topdata['result'], test_size=0.21)
len(x_train), len(x_test), len(y_train), len(y_test)

(22980, 6109, 22980, 6109)

In [394]:
# 로지스틱 회귀분석으로 모델학습
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear', random_state=1, C=0.01, )
model.fit(x_train, y_train)
pred = model.predict(x_test)
pred

array([1, 0, 1, ..., 0, 1, 1], dtype=int64)

In [395]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
# 모델의 정확도,정밀도,재현율 
accuracy_score(pred,y_test),precision_score(pred,y_test),recall_score(pred,y_test)

(0.6460959240464887, 0.6349822179114128, 0.655321988655322)

In [125]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=500, max_depth=5)
rfc.fit(x_train, y_train)
pred = rfc.predict(x_test)
accuracy_score(pred,y_test)

0.6271139832256291

In [25]:
from xgboost import XGBClassifier
rfc = XGBClassifier(n_estimators=500, max_depth=5, learning_rate=0.01)
rfc.fit(x_train, y_train)
pred = rfc.predict(x_test)
accuracy_score(pred,y_test)

0.6261225417756053

In [26]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train2 = scaler.fit_transform(x_train)
x_test2 = scaler.fit_transform(x_test)

rfc = XGBClassifier(n_estimators=500, max_depth=5, learning_rate=0.01)
rfc.fit(x_train2, y_train)
pred = rfc.predict(x_test2)
accuracy_score(pred,y_test)

0.6235080140957144

In [27]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=500, max_depth=6)
rfc.fit(x_train2, y_train)
pred = rfc.predict(x_test2)
accuracy_score(pred,y_test)

0.6279413436398772